This notebook collects all distinct `drug_concept_id`s in the MongoDB and maps them to drug names in the MySQL `concept` table. It then creates a Javascript file to populate the drug selection box.

In [1]:
import sys
import pymongo
import pymysql

In [2]:
# Mongo login
EXTRACTED_DIR = './results/extracted'
REFERNCE_DIR = './reference'
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017

print >> sys.stderr, "Loading password from ../nsides-mongo.cnf..."
MONGODB_HOST, MONGODB_UN, MONGODB_PW = open('../nsides-mongo.cnf').read().strip().split('\n')

# print >> sys.stderr, "Reading the 'nsides' mongodb at %s:%s" % (MONGODB_HOST, MONGODB_PORT)

client = pymongo.MongoClient('mongodb://%s:%s@%s:%s/admin' % (MONGODB_UN, MONGODB_PW, MONGODB_HOST, MONGODB_PORT))
db = client.nsides
estimates = db.estimates

print >> sys.stderr, "%s" % db.collection_names()
print >> sys.stderr, "%s" % str(estimates.count())

Loading password from ../nsides-mongo.cnf...
[u'estimates']
13298882


In [3]:
# MySQL login
print "Connecting to MySQL database"

conn = pymysql.connect(read_default_file='../nsides.cnf',
                        db='ebdb',
                        charset='utf8mb4',
                        cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

Connecting to MySQL database


In [4]:
# Collect all distinct drug_concept_ids in MongoDB
all_drugs = db.estimates.distinct('rxnorm')
print len(all_drugs), "drugs total"

1859 drugs total


In [5]:
# Collect drug names from MySQL
drugname2concept_id = dict()

SQL = '''select concept_id, concept_name
         from concept
         where concept_id in %s''' %str(tuple(all_drugs))
cur.execute(SQL)
results = cur.fetchall()

for result in results:
    drug_concept_id = result[u'concept_id']
    drugname = result[u'concept_name'].encode('ascii','ignore').capitalize()
    
    drugname2concept_id[drugname] = drug_concept_id

print len(drugname2concept_id), "drugs mapped"

1859 drugs mapped


In [6]:
# Write to file
f = open('../index/js/drugs.js', 'w')
f.write('var drugs = [\n')
for drugname in sorted(drugname2concept_id.keys()):
    f.write('{ value: "%d", label: "%s" },\n' %(drugname2concept_id[drugname], drugname))
    
f.write('];')
f.close()

In [7]:
cur.close()
conn.close()